# Preprocessing OA data

In [4]:
def add_lsoa_codes_from_oa(dataset_path, oa_column_name, lookup_path,
                           lookup_oa_col='OA21CD', lookup_lsoa_col='LSOA21CD',
                           output_dir="data/preprocessed"):
    """
    Add LSOA codes to a dataset using OA codes and a lookup table.

    Parameters:
    dataset_path (str): Path to your dataset containing OA codes
    oa_column_name (str): Name of the column containing OA codes in your dataset
    lookup_path (str): Path to the OA to LSOA lookup file
    lookup_oa_col (str): Column name in lookup table containing OA codes
    lookup_lsoa_col (str): Column name in lookup table containing LSOA codes
    output_dir (str): Directory to save the processed dataset

    Returns:
    pd.DataFrame: Dataset with added LSOA codes
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Generate timestamp for unique filenames
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Extract filename for reporting
    base_filename = os.path.basename(dataset_path).split('.')[0]

    print(f"Processing dataset: {dataset_path}")

    # Load your dataset
    if dataset_path.endswith('.csv'):
        df = pd.read_csv(dataset_path, low_memory=False)
    elif dataset_path.endswith(('.xls', '.xlsx')):
        df = pd.read_excel(dataset_path)
    else:
        raise ValueError("Unsupported file format. Please use CSV or Excel.")

    print(f"Dataset loaded with shape: {df.shape}")

    # Load the lookup table
    if lookup_path.endswith('.csv'):
        lookup_df = pd.read_csv(lookup_path, low_memory=False)
    elif lookup_path.endswith(('.xls', '.xlsx')):
        lookup_df = pd.read_excel(lookup_path)
    else:
        raise ValueError("Unsupported lookup file format. Please use CSV or Excel.")

    print(f"Lookup table loaded with shape: {lookup_df.shape}")

    # Identify column names in the lookup table
    print("Lookup table column names:")
    for col in lookup_df.columns:
        print(f"- {col}")

    # Use the explicitly provided column names for OA and LSOA
    if lookup_oa_col not in lookup_df.columns:
        raise ValueError(f"OA column '{lookup_oa_col}' not found in lookup table.")

    if lookup_lsoa_col not in lookup_df.columns:
        raise ValueError(f"LSOA column '{lookup_lsoa_col}' not found in lookup table.")

    print(f"Using lookup columns: {lookup_oa_col} -> {lookup_lsoa_col}")

    # Standardize column names in your dataset
    df.columns = [col.lower().replace(' ', '_') for col in df.columns]
    oa_column_name = oa_column_name.lower().replace(' ', '_')

    # Check if OA column exists in your dataset
    if oa_column_name not in df.columns:
        raise ValueError(f"OA column '{oa_column_name}' not found in dataset. Available columns: {df.columns.tolist()}")

    # Print first few values of the OA column to check format
    print(f"First 5 values of the OA column ({oa_column_name}):")
    for val in df[oa_column_name].head(5):
        print(f"- {val}")

    # Clean OA codes in your dataset
    df[oa_column_name] = df[oa_column_name].astype(str).str.strip()

    # Create a mapping dictionary from OA to LSOA
    oa_to_lsoa = dict(zip(lookup_df[lookup_oa_col], lookup_df[lookup_lsoa_col]))

    # Print first few items in mapping dictionary
    print("First 5 items in OA to LSOA mapping:")
    for i, (oa, lsoa) in enumerate(list(oa_to_lsoa.items())[:5]):
        print(f"- {oa} -> {lsoa}")

    # Add LSOA codes to your dataset
    df['lsoa_code'] = df[oa_column_name].map(oa_to_lsoa)

    # Count successful mappings
    mapped_count = df['lsoa_code'].notna().sum()

    print(f"Successfully mapped {mapped_count}/{len(df)} records ({mapped_count/len(df):.2%})")

    # Analyze unmapped OAs if any
    if mapped_count < len(df):
        unmapped_oas = df[df['lsoa_code'].isna()][oa_column_name].unique()
        print(f"Found {len(unmapped_oas)} unique unmapped OA codes")

        # Display a sample of unmapped OAs
        print("Sample of unmapped OA codes:")
        for oa in unmapped_oas[:5]:
            print(f"- {oa}")

        # Save unmapped OAs to a file for investigation
        with open(f"{output_dir}/{base_filename}_{timestamp}_unmapped_oas.txt", "w") as f:
            f.write("Unmapped OA codes:\n")
            for oa in unmapped_oas[:100]:  # Limit to first 100 for brevity
                f.write(f"- {oa}\n")
            if len(unmapped_oas) > 100:
                f.write(f"... and {len(unmapped_oas) - 100} more")

    # Save the dataset with added LSOA codes
    output_path = f"{output_dir}/{base_filename}_with_lsoa.csv"
    df.to_csv(output_path, index=False)
    print(f"Dataset with LSOA codes saved to: {output_path}")

    # Create a basic report
    with open(f"{output_dir}/{base_filename}_{timestamp}_mapping_report.txt", "w") as f:
        f.write(f"OA to LSOA Mapping Report\n")
        f.write(f"=======================\n\n")
        f.write(f"Dataset: {dataset_path}\n")
        f.write(f"Lookup table: {lookup_path}\n")
        f.write(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

        f.write(f"Mapping Statistics:\n")
        f.write(f"-----------------\n")
        f.write(f"Total records: {len(df)}\n")
        f.write(f"Records with mapped LSOA codes: {mapped_count} ({mapped_count/len(df):.2%})\n")
        f.write(f"Records with unmapped LSOA codes: {len(df) - mapped_count} ({(len(df) - mapped_count)/len(df):.2%})\n\n")

        if mapped_count < len(df):
            f.write(f"Unmapped OAs Investigation:\n")
            f.write(f"-------------------------\n")
            f.write(f"See the file '{base_filename}_{timestamp}_unmapped_oas.txt' for a list of unmapped OA codes.\n\n")

            f.write(f"Possible reasons for unmapped OAs:\n")
            f.write(f"1. OA codes in your dataset might be from a different Census year than the lookup table\n")
            f.write(f"2. OA codes might contain typos or formatting issues\n")
            f.write(f"3. OA codes might be from outside Greater Manchester\n\n")

            f.write(f"Recommendations:\n")
            f.write(f"1. Verify the OA codes in your dataset\n")
            f.write(f"2. Check if you need a different lookup table version\n")
            f.write(f"3. Consider using a fuzzier matching approach for partially incorrect codes\n")

    print(f"Mapping report saved to: {output_dir}/{base_filename}_{timestamp}_mapping_report.txt")

    return df, {
        'output_file': output_path,
        'report_file': f"{output_dir}/{base_filename}_{timestamp}_mapping_report.txt"
    }

### fixed_coverage

In [6]:
# Example usage
dataset_path = "data/rawdata/communications/fixed_coverage.csv"
oa_column_name = "output_area"  # Replace with your actual OA column name
lookup_path = "data/gm_oa_lookup.xlsx"

# Run the function with explicitly specified column names
result_df, output_files = add_lsoa_codes_from_oa(
    dataset_path=dataset_path,
    oa_column_name=oa_column_name,
    lookup_path=lookup_path,
    lookup_oa_col='OA21CD',  # Explicitly specify the OA column in lookup
    lookup_lsoa_col='LSOA21CD',  # Explicitly specify the LSOA column in lookup
    output_dir="data/preprocessed/communications"
)

Processing dataset: data/rawdata/communications/fixed_coverage.csv
Dataset loaded with shape: (238656, 37)
Lookup table loaded with shape: (8966, 12)
Lookup table column names:
- OA21CD
- LSOA21CD
- LSOA21NM
- MSOA21CD
- MSOA21NM
- LEP22CD1
- LEP22NM1
- LEP22CD2
- LEP22NM2
- LAD22CD
- LAD22NM
- ObjectId
Using lookup columns: OA21CD -> LSOA21CD
First 5 values of the OA column (output_area):
- E00000001
- E00000003
- E00000005
- E00000007
- E00000010
First 5 items in OA to LSOA mapping:
- E00024150 -> E01004772
- E00024305 -> E01004799
- E00024203 -> E01004782
- E00024255 -> E01004787
- E00024151 -> E01004772
Successfully mapped 8963/238656 records (3.76%)
Found 229693 unique unmapped OA codes
Sample of unmapped OA codes:
- E00000001
- E00000003
- E00000005
- E00000007
- E00000010
Dataset with LSOA codes saved to: data/preprocessed/communications/fixed_coverage_with_lsoa.csv
Mapping report saved to: data/preprocessed/communications/fixed_coverage_20250409_003658_mapping_report.txt


In [7]:
df = pd.read_csv("data/preprocessed/communications/fixed_coverage_with_lsoa.csv")
df.head()

/var/folders/7h/kpwlmbts6qvbvpgj641mfzjr0000gn/T/ipykernel_91799/2201733779.py:1: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/preprocessed/communications/fixed_coverage_with_lsoa.csv")


,output_area,all_premises,all_matched_premises,sfbb_availability_(%_premises),ufbb_(100mbit/s)_availability_(%_premises),ufbb_availability_(%_premises),%_of_premises_unable_to_receive_2mbit/s,%_of_premises_unable_to_receive_5mbit/s,%_of_premises_unable_to_receive_10mbit/s,%_of_premises_unable_to_receive_30mbit/s,...,%_of_premises_with_2<5mbit/s_download_speed,%_of_premises_with_5<10mbit/s_download_speed,%_of_premises_with_10<30mbit/s_download_speed,number_of_premises_with_30<300mbit/s_download_speed,number_of_premises_with_>=300mbit/s_download_speed,number_of_premises_with_0<2mbit/s_download_speed,number_of_premises_with_2<5mbit/s_download_speed,number_of_premises_with_5<10mbit/s_download_speed,number_of_premises_with_10<30mbit/s_download_speed,lsoa_code
0,E00000001,119,119,99.2,99.2,99.2,0.0,0.0,0.0,0.8,...,0.0,0.0,0.8,0,118,0,0,0,1,NaN
1,E00000003,127,127,100.0,100.0,100.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,127,0,0,0,0,NaN
2,E00000005,78,78,98.7,98.7,98.7,0.0,0.0,0.0,1.3,...,0.0,0.0,1.3,0,77,0,0,0,1,NaN
3,E00000007,151,150,14.6,14.6,14.6,0.0,0.0,0.0,84.8,...,0.0,0.0,84.8,0,22,0,0,0,128,NaN
4,E00000010,160,160,100.0,99.4,99.4,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1,159,0,0,0,0,NaN


In [8]:
df.columns

Index(['output_area', 'all_premises', 'all_matched_premises',
       'sfbb_availability_(%_premises)',
       'ufbb_(100mbit/s)_availability_(%_premises)',
       'ufbb_availability_(%_premises)',
       '%_of_premises_unable_to_receive_2mbit/s',
       '%_of_premises_unable_to_receive_5mbit/s',
       '%_of_premises_unable_to_receive_10mbit/s',
       '%_of_premises_unable_to_receive_30mbit/s',
       'gigabit_availability_(%_premises)', '%_of_premises_below_the_uso',
       '%_of_premises_with_nga',
       '%_of_premises_able_to_receive_decent_broadband_from_fwa',
       'number_of_premises_with_sfbb_availability',
       'number_of_premises_with_ufbb_(100mbit/s)_availability',
       'number_of_premises_with_ufbb_availability',
       'number_of_premises_unable_to_receive_2mbit/s',
       'number_of_premises_unable_to_receive_5mbit/s',
       'number_of_premises_unable_to_receive_10mbit/s',
       'number_of_premises_unable_to_receive_30mbit/s',
       'number_of_premises_with_gigab

In [9]:
# 1. Filter rows where lsoa_code is not null
df_filtered = df[df['lsoa_code'].notnull()]
df_filtered.head()

,output_area,all_premises,all_matched_premises,sfbb_availability_(%_premises),ufbb_(100mbit/s)_availability_(%_premises),ufbb_availability_(%_premises),%_of_premises_unable_to_receive_2mbit/s,%_of_premises_unable_to_receive_5mbit/s,%_of_premises_unable_to_receive_10mbit/s,%_of_premises_unable_to_receive_30mbit/s,...,%_of_premises_with_2<5mbit/s_download_speed,%_of_premises_with_5<10mbit/s_download_speed,%_of_premises_with_10<30mbit/s_download_speed,number_of_premises_with_30<300mbit/s_download_speed,number_of_premises_with_>=300mbit/s_download_speed,number_of_premises_with_0<2mbit/s_download_speed,number_of_premises_with_2<5mbit/s_download_speed,number_of_premises_with_5<10mbit/s_download_speed,number_of_premises_with_10<30mbit/s_download_speed,lsoa_code
22579,E00024141,144,144,100.0,99.3,99.3,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1,143,0,0,0,0,E01004771
22580,E00024142,120,120,99.2,99.2,99.2,0.0,0.0,0.0,0.8,...,0.0,0.0,0.8,0,119,0,0,0,1,E01004770
22581,E00024143,128,128,100.0,100.0,100.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,128,0,0,0,0,E01004767
22582,E00024144,130,130,100.0,100.0,100.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,130,0,0,0,0,E01004769
22583,E00024145,133,133,100.0,100.0,100.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,133,0,0,0,0,E01004771


In [15]:
df_filtered.to_csv('data/preprocessed/communications/fixed_coverage_with_lsoa2.csv', index=False)

### fixed_performance

In [16]:
# Example usage
dataset_path = "data/rawdata/communications/fixed_performance.csv"
oa_column_name = "output_area"  # Replace with your actual OA column name
lookup_path = "data/gm_oa_lookup.xlsx"

# Run the function with explicitly specified column names
result_df, output_files = add_lsoa_codes_from_oa(
    dataset_path=dataset_path,
    oa_column_name=oa_column_name,
    lookup_path=lookup_path,
    lookup_oa_col='OA21CD',  # Explicitly specify the OA column in lookup
    lookup_lsoa_col='LSOA21CD',  # Explicitly specify the LSOA column in lookup
    output_dir="data/preprocessed/communications"
)

Processing dataset: data/rawdata/communications/fixed_performance.csv
Dataset loaded with shape: (238627, 31)
Lookup table loaded with shape: (8966, 12)
Lookup table column names:
- OA21CD
- LSOA21CD
- LSOA21NM
- MSOA21CD
- MSOA21NM
- LEP22CD1
- LEP22NM1
- LEP22CD2
- LEP22NM2
- LAD22CD
- LAD22NM
- ObjectId
Using lookup columns: OA21CD -> LSOA21CD
First 5 values of the OA column (output_area):
- E00000001
- E00000003
- E00000005
- E00000007
- E00000010
First 5 items in OA to LSOA mapping:
- E00024150 -> E01004772
- E00024305 -> E01004799
- E00024203 -> E01004782
- E00024255 -> E01004787
- E00024151 -> E01004772
Successfully mapped 8962/238627 records (3.76%)
Found 229665 unique unmapped OA codes
Sample of unmapped OA codes:
- E00000001
- E00000003
- E00000005
- E00000007
- E00000010
Dataset with LSOA codes saved to: data/preprocessed/communications/fixed_performance_with_lsoa.csv
Mapping report saved to: data/preprocessed/communications/fixed_performance_20250409_004658_mapping_report.t

In [17]:
df2 = pd.read_csv("data/preprocessed/communications/fixed_performance_with_lsoa.csv")
df2.head()

/var/folders/7h/kpwlmbts6qvbvpgj641mfzjr0000gn/T/ipykernel_91799/364565618.py:1: DtypeWarning: Columns (24,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("data/preprocessed/communications/fixed_performance_with_lsoa.csv")


,output_area,median_download_speed_(mbit/s),average_download_speed_(mbit/s),maximum_download_speed_(mbit/s),average_download_speed_(mbit/s)_for_lines_<_10mbit/s,average_download_speed_(mbit/s)_for_lines_10<30mbit/s,average_download_speed_(mbit/s)_for_lines_30<300mbit/s,average_download_speed_(mbit/s)_for_sfbb_lines,average_download_speed_(mbit/s)_for_ufbb_lines,median_upload_speed_(mbit/s),...,number_of_connections_>=300_mbit/s_(number_of_lines),number_of_connections_>=30_mbit/s_(number_of_lines),average_data_usage_(gb),median_data_usage_(gb),average_data_usage_(gb)_for_lines_<10mbits,average_data_usage_(gb)_for_lines_10<30mbit/s,average_data_usage_(gb)_for_lines_30<300mbit/s,average_data_usage_(gb)_for_sfbb_lines,average_data_usage_(gb)_for_ufbb_lines,lsoa_code
0,E00000001,80.0,236.7,1000.0,2.0,11.5,101.5,242.4,863.6,50.0,...,22.0,119.0,228.0,95.075,11.0,109.0,242.0,242.0,NaN,NaN
1,E00000003,79.7,145.7,1000.0,6.0,23.5,96.7,152.8,842.9,50.0,...,7.0,93.0,251.0,175.550,176.0,28.0,268.0,268.0,NaN,NaN
2,E00000005,79.7,131.6,1000.0,4.7,13.5,91.5,142.2,680.0,50.0,...,5.0,58.0,212.0,138.945,49.0,22.0,247.0,247.0,NaN,NaN
3,E00000007,9.7,20.0,1000.0,6.1,15.4,97.1,210.0,1000.0,1.1,...,1.0,8.0,151.0,27.345,47.0,226.0,214.0,339.0,1089.0,NaN
4,E00000010,79.7,124.0,1000.0,8.5,19.2,79.6,133.6,790.3,20.0,...,6.0,79.0,277.0,141.450,92.0,234.0,283.0,283.0,NaN,NaN


In [18]:
# 1. Filter rows where lsoa_code is not null
df2_filtered = df2[df2['lsoa_code'].notnull()]
df2_filtered.head()

,output_area,median_download_speed_(mbit/s),average_download_speed_(mbit/s),maximum_download_speed_(mbit/s),average_download_speed_(mbit/s)_for_lines_<_10mbit/s,average_download_speed_(mbit/s)_for_lines_10<30mbit/s,average_download_speed_(mbit/s)_for_lines_30<300mbit/s,average_download_speed_(mbit/s)_for_sfbb_lines,average_download_speed_(mbit/s)_for_ufbb_lines,median_upload_speed_(mbit/s),...,number_of_connections_>=300_mbit/s_(number_of_lines),number_of_connections_>=30_mbit/s_(number_of_lines),average_data_usage_(gb),median_data_usage_(gb),average_data_usage_(gb)_for_lines_<10mbits,average_data_usage_(gb)_for_lines_10<30mbit/s,average_data_usage_(gb)_for_lines_30<300mbit/s,average_data_usage_(gb)_for_sfbb_lines,average_data_usage_(gb)_for_ufbb_lines,lsoa_code
22590,E00024141,125.0,206.9,1000.0,8.0,20.2,142.5,267.9,591.7,10.0,...,24.0,86.0,482,338.555,NaN,347.0,534.0,538.0,553.0,E01004771
22591,E00024142,125.0,233.8,1000.0,NaN,19.4,153.0,328.4,630.0,10.0,...,25.0,68.0,372,298.930,NaN,315.0,372.0,408.0,500.0,E01004770
22592,E00024143,125.0,222.8,1000.0,NaN,21.6,148.0,268.5,653.2,10.0,...,21.0,88.0,1715,334.770,NaN,490.0,413.0,2039.0,7786.0,E01004767
22593,E00024144,79.2,176.1,1000.0,10.0,16.8,101.4,187.0,603.3,14.3,...,15.0,88.0,370,223.260,4.0,339.0,403.0,376.0,169.0,E01004769
22594,E00024145,125.0,188.0,1000.0,7.0,19.4,115.6,216.8,547.8,10.0,...,22.0,94.0,483,348.650,198.0,180.0,532.0,546.0,608.0,E01004771


In [20]:
print(df2_filtered.isnull().sum())

output_area                                                  0
median_download_speed_(mbit/s)                               0
average_download_speed_(mbit/s)                              0
maximum_download_speed_(mbit/s)                              0
average_download_speed_(mbit/s)_for_lines_<_10mbit/s      1963
average_download_speed_(mbit/s)_for_lines_10<30mbit/s      513
average_download_speed_(mbit/s)_for_lines_30<300mbit/s      10
average_download_speed_(mbit/s)_for_sfbb_lines              10
average_download_speed_(mbit/s)_for_ufbb_lines             540
median_upload_speed_(mbit/s)                                 0
average_upload_speed_(mbit/s)                                3
maximum_upload_speed_(mbit/s)                                3
average_upload_speed_(mbit/s)_for_lines_<_10mbit/s        1963
average_upload_speed_(mbit/s)_for_lines_10<30mbit/s        513
average_upload_speed_(mbit/s)_for_lines_30<300mbit/s        10
average_upload_speed_(mbit/s)_for_sfbb_lines           

In [22]:
df2_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8962 entries, 22590 to 169820
Data columns (total 32 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   output_area                                             8962 non-null   object 
 1   median_download_speed_(mbit/s)                          8962 non-null   float64
 2   average_download_speed_(mbit/s)                         8962 non-null   float64
 3   maximum_download_speed_(mbit/s)                         8962 non-null   float64
 4   average_download_speed_(mbit/s)_for_lines_<_10mbit/s    6999 non-null   float64
 5   average_download_speed_(mbit/s)_for_lines_10<30mbit/s   8449 non-null   float64
 6   average_download_speed_(mbit/s)_for_lines_30<300mbit/s  8952 non-null   float64
 7   average_download_speed_(mbit/s)_for_sfbb_lines          8952 non-null   float64
 8   average_download_speed_(mbit/s)_

In [28]:
df2_filtered = df2_filtered.fillna(0)

In [31]:
df2_filtered.to_csv('data/preprocessed/communications/fixed_performance_with_lsoa2.csv', index=False)